In [1]:
from pymongo import MongoClient
import configparser
import pandas as pd
import matplotlib.pyplot as plt

from Levenshtein import distance

In [2]:
# Read Configurations to connect to MongoDB
config = configparser.ConfigParser()
config.read('mongo.cnf')
usr = config.get('mongodb_local', 'username')
pwd = config.get('mongodb_local', 'password')
host = config.get('mongodb_local', 'host')
port = config.get('mongodb_local', 'port')
db = config.get('mongodb_local', 'db')
gspx_collection = config.get('mongodb_local', 'gspx_collection')
api_collection = config.get('mongodb_local', 'api_collection')

In [3]:
# Connect to MongoDB and access gspx and guardian API collections
uri = f'mongodb://{usr}:{pwd}@{host}:{port}/?authMechanism=DEFAULT'
cluster = MongoClient(uri)
database = cluster[db]
guardian_gspx = database[gspx_collection]
guardian_api = database[api_collection]

In [4]:
# gspx guardian1 year range: 1792 - 2013
# guardian api year range: 1999 - 2022
df_gspx = pd.json_normalize(list(guardian_gspx.find()))
df_api = pd.json_normalize(list(guardian_api.find()))

# a little sanitation work
df_gspx = df_gspx.rename(columns={'Title': 'title'})

In [44]:
# Separate event year and month from eventTimeStamp
df_gspx['eventTimeStamp'] = pd.to_datetime(df_gspx['eventTimeStamp'])
# Avoids IntCastingNaError by casting into Int64
df_gspx['eventYear'] = df_gspx['eventTimeStamp'].dt.year.astype('Int64')
df_gspx['eventMonth'] = df_gspx['eventTimeStamp'].dt.month.astype('Int64')

df_api['eventTimeStamp'] = pd.to_datetime(df_api['fields.newspaperEditionDate'])
df_api['eventYear'] = df_api['eventTimeStamp'].dt.year.astype('Int64')
df_api['eventMonth'] = df_api['eventTimeStamp'].dt.month.astype('Int64')

# Within gspx guardian1 database:
# ProQuest year range: 1792 - 2003
# LexisNexis year range: 2004 - 2013
df_proquest = df_gspx[df_gspx['pages']=='ProQuest']
df_lexisnexis = df_gspx[df_gspx['pages']=='LexisNexis']

# trim two dataframes so ProQuest: 1999 - 2003 & API: 1999 - 2013
df_proquest = df_proquest[df_proquest['eventYear'] >= 1999]
df_api = df_api[df_api['eventYear'] <= 2003]

In [57]:
# gspx dataframe structure
df_gspx

,_id,randomOrder,articleID,title,Abstract,pages,year_old,month_old,day_old,DocumentURL,...,training,eventTimeStamp,day,month,year,liveUpdate,eventYear,eventMonth,LevenshteinMatchingTitle,LevenshteinDistance
0,6480a03f85c16bfebbe60d17,1,475799244,"THE ""FACTS OF LIFE"" AGITATION",The Manchester Guardian (1901-1959),ProQuest,1914,0,"Apr 24, 1914",http://search.proquest.com/docview/475799244?a...,...,0,1914-04-24,24,4,1914,0,1914,4,<NA>,NaN
1,6480a03f85c16bfebbe60d18,2,187630660,Drugs crackdown partly to blame for prison riot,The Guardian (1959-2003),ProQuest,1995,0,"Jan 19, 1995",http://search.proquest.com/docview/187630660?a...,...,0,1995-01-19,19,1,1995,0,1995,1,<NA>,NaN
2,6480a03f85c16bfebbe60d19,3,0,Anaheim officials scramble to contain public a...,Guardian.com,LexisNexis,2012,7,"July 26, 2012 Thursday",http://socweb.soc.jhu.edu/gsp/guardian/The_Gua...,...,0,2012-07-26,26,7,2012,0,2012,7,<NA>,NaN
3,6480a03f85c16bfebbe60d1a,4,185173777,Women talking: about Vietnam and protest,The Guardian (1959-2003),ProQuest,1965,0,"Nov 8, 1965",http://search.proquest.com/docview/185173777?a...,...,0,1965-11-08,8,11,1965,0,1965,11,<NA>,NaN
4,6480a03f85c16bfebbe60d1b,5,0,China: Leader of province hit by ethnic riots ...,The Guardian - Final Edition,LexisNexis,2010,4,"April 26, 2010 Monday",http://socweb.soc.jhu.edu/gsp/guardian/The_Gua...,...,0,2010-04-26,26,4,2010,0,2010,4,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59659,6480a04685c16bfebbe6f622,0,0,International cartoon protests spread,Guardian.com,LexisNexis,2006,2,38754,http://socweb.soc.jhu.edu/gsp/guardian/The_Gua...,...,0,NaT,0,0,0,0,<NA>,<NA>,<NA>,NaN
59660,6480a04685c16bfebbe6f623,0,0,International cartoon protests spread,Guardian.com,LexisNexis,2006,2,38754,http://socweb.soc.jhu.edu/gsp/guardian/The_Gua...,...,0,NaT,0,0,0,0,<NA>,<NA>,<NA>,NaN
59661,6480a04685c16bfebbe6f624,0,0,International cartoon protests spread,Guardian.com,LexisNexis,2006,2,38754,http://socweb.soc.jhu.edu/gsp/guardian/The_Gua...,...,0,NaT,0,0,0,0,<NA>,<NA>,<NA>,NaN
59662,6480a04685c16bfebbe6f625,0,478076344,TROUBLE IN FRENCH INDOCHINA,The Manchester Guardian (1901-1959),ProQuest,1930,0,7-Jun-30,http://search.proquest.com/docview/478076344?a...,...,0,NaT,0,0,0,0,<NA>,<NA>,<NA>,NaN


In [58]:
# api dataframe structure (flattened json)
df_api

,_id,title,url,guardianid,_type,section.id,section.name,fields.headline,fields.standfirst,fields.trailText,...,fields.sensitive,apiresult.fields.sensitive,fields.contributorBio,apiresult.fields.contributorBio,eventTimeStamp,eventYear,eventMonth,LevenshteinMatchingTitle,LevenshteinMatchingDatetime,LevenshteinDistance
0,647f81bcbfa4a0c3c9318dc5,Artists' revolt as Dirty Digger buys $7m love ...,https://www.theguardian.com/world/1999/jan/31/...,world/1999/jan/31/theobserver1,GuardianArticle,world,World news,Artists' revolt as Dirty Digger buys $7m love ...,By Edward Helmore and Melinda Wittstock in New...,Artists in one of New York's most fashionable ...,...,NaN,NaN,NaN,NaN,1999-01-31 00:00:00+00:00,1999,1,Religious riots rock Indonesian island,1999-01-21,45.0
1,647f81bcbfa4a0c3c9318dc6,Aid chaos leads to protest riots,https://www.theguardian.com/world/1999/jan/30/7,world/1999/jan/30/7,GuardianArticle,world,World news,Aid chaos leads to protest riots,"Fresh rioting broke out yesterday in Armenia, ...",The unrest began when an elderly woman fainted...,...,NaN,NaN,NaN,NaN,1999-01-30 00:00:00+00:00,1999,1,Aid chaos leads to protest riots,1999-01-30,0.0
2,647f81bcbfa4a0c3c9318dc7,Boateng in clash over child abuse,https://www.theguardian.com/uk/1999/jan/28/ala...,uk/1999/jan/28/alantravis,GuardianArticle,uk-news,UK news,Boateng in clash over child abuse,NaN,A Storm of protest broke over the head of the...,...,NaN,NaN,NaN,NaN,1999-01-28 00:00:00+00:00,1999,1,Boateng in clash over child abuse,1999-01-28,0.0
3,647f81bcbfa4a0c3c9318dc8,Small shops protest at £90 watchdog levy,https://www.theguardian.com/news/1999/jan/27/f...,news/1999/jan/27/food.foodanddrink3,GuardianArticle,food,Food,Small shops protest at £90 watchdog levy,NaN,Small business leaders reacted angrily tonight...,...,NaN,NaN,NaN,NaN,1999-01-27 00:00:00+00:00,1999,1,Rebel without a suitcase,1999-01-06,30.0
4,647f81bcbfa4a0c3c9318dc9,MPs back gay consent age of 16 but face new cl...,https://www.theguardian.com/politics/1999/jan/...,politics/1999/jan/26/uk.politicalnews1,GuardianArticle,politics,Politics,MPs back gay consent age of 16 but face new cl...,NaN,MPs last night voted to create equality before...,...,NaN,NaN,NaN,NaN,1999-01-26 00:00:00+00:00,1999,1,Internet on line for new revolution,1999-01-10,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2608,647f829f2f5adb71e302196e,Roman friars unnerved by nun's live-in protest,https://www.theguardian.com/world/2003/dec/13/...,world/2003/dec/13/italy.johnhooper,GuardianArticle,world,World news,Roman friars unnerved by nun's live-in protest,NaN,A nun who refuses to leave her cell in a Rome ...,...,NaN,NaN,NaN,NaN,2003-12-13 00:00:00+00:00,2003,12,Court win in ships protest,2003-12-16,30.0
2609,647f829f2f5adb71e3021970,Fishing protest leaves booze cruisers stranded,https://www.theguardian.com/environment/2003/d...,environment/2003/dec/10/fishing.food,GuardianArticle,environment,Environment,Fishing protest leaves booze cruisers stranded,NaN,<p>Christmas booze cruisers were today left st...,...,NaN,NaN,NaN,NaN,2003-12-10 00:00:00+00:00,2003,12,Tories launch pre-budget strike,2003-12-10,32.0
2662,647f829f2f5adb71e3021972,Labour and Lib Dems in tax clash,https://www.theguardian.com/politics/2003/dec/...,politics/2003/dec/08/liberaldemocrats.uk,GuardianArticle,politics,Politics,Labour and Lib Dems in tax clash,NaN,Labour and the Liberal Democrats battled over ...,...,NaN,NaN,NaN,NaN,2003-12-08 00:00:00+00:00,2003,12,Court win in ships protest,2003-12-16,23.0
2663,647f829f2f5adb71e3021974,Blunkett denies leading fees revolt,https://www.theguardian.com/politics/2003/dec/...,politics/2003/dec/05/publicservices.uk1,GuardianArticle,politics,Politics,Blunkett denies leading fees revolt,"<A HREF=""http://edm.ais.co.uk/weblink/html/mot...","The home secretary, David Blunkett, has denied...",...,NaN,NaN,NaN,NaN,2003-12-05 00:00:00+00:00,2003,12,Court win in ships protest,2003-12-16,25.0


In [35]:
# Function to calculate resemblance score using Levenshtein Distance
# The lower the score, the more similar the two strings are
# Can add other distance functions in the future
def calculate_levenshtein_distance(row_a, row_b):
    return distance(row_a['title'], row_b['title'])

In [56]:
df_proquest['LevenshteinMatchingTitle'] = pd.Series(dtype='string')
df_proquest['LevenshteinDistance'] = pd.Series(dtype='float')
df_lexisnexis['LevenshteinMatchingTitle'] = pd.Series(dtype='string')
df_lexisnexis['LevenshteinDistance'] = pd.Series(dtype='float')
df_gspx['LevenshteinMatchingTitle'] = pd.Series(dtype='string')
df_gspx['LevenshteinDistance'] = pd.Series(dtype='float')

# Only matching with articles from the same month for testing (computational speed)
for i, row_proquest in df_proquest.iterrows():
    selected_rows = df_api[(df_api['eventYear'] == row_proquest['eventYear']) & (df_api['eventMonth'] == row_proquest['eventMonth'])]
    best_match_index = min(range(len(selected_rows)), key=lambda j: calculate_levenshtein_distance(row_proquest, selected_rows.iloc[j]))
    df_proquest.loc[i, 'LevenshteinMatchingTitle'] = selected_rows.iloc[best_match_index]['title']
    df_proquest.loc[i, 'LevenshteinMatchingDatetime'] = selected_rows.iloc[best_match_index]['fields.newspaperEditionDate']
    df_proquest.loc[i, 'LevenshteinDistance'] = calculate_levenshtein_distance(row_proquest, selected_rows.iloc[best_match_index])

C:\Users\Adrian\AppData\Local\Temp\ipykernel_31352\2172277439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lexisnexis['LevenshteinMatchingTitle'] = pd.Series(dtype='string')
C:\Users\Adrian\AppData\Local\Temp\ipykernel_31352\2172277439.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lexisnexis['LevenshteinDistance'] = pd.Series(dtype='float')


In [55]:
'''
1999 - 2003

For ProQuest guardian1 database (2640):
465 articles have been matched with an article with Levenshtein Distance less than or equal 10 (match)
625 articles have been matched with an article with Levenshtein Distance between 10 and 20 (doubtful)
1550 articles have been matched with an article with Levenshtein Distance greater than 20 (no match)

For Guardian API scraped database (2123):
421 articles have been matched with an article with Levenshtein Distance less than or equal 10 (match)
344 articles have been matched with an article with Levenshtein Distance between 10 and 20 (doubtful)
1358 articles have been matched with an article with Levenshtein Distance greater than 20 (no match)
'''

'\nFor ProQuest guardian1 database (2640):\n465 articles have been matched with an article with Levenshtein Distance less than or equal 10 (match)\n625 articles have been matched with an article with Levenshtein Distance between 10 and 20 (doubtful)\n1550 articles have been matched with an article with Levenshtein Distance greater than 20 (no match)\n\nFor Guardian API scraped database (2123):\n421 articles have been matched with an article with Levenshtein Distance less than or equal 10 (match)\n344 articles have been matched with an article with Levenshtein Distance between 10 and 20 (doubtful)\n1358 articles have been matched with an article with Levenshtein Distance greater than 20 (no match)\n'

In [ ]:
'''
Below is the code that counts events by year and month
'''

In [111]:
proquest_datetime = pd.to_datetime(df_proquest['eventTimeStamp'])
# Extract year and month from each datetime object
year = proquest_datetime.dt.year
month = proquest_datetime.dt.month
df = pd.DataFrame({'Year': year, 'Month': month})
pivot_table = df.groupby(['Year', 'Month']).size().unstack(fill_value=0)
pivot_table = pivot_table.reindex(sorted(pivot_table.columns, reverse=True), axis=1).sort_index(ascending=False)
pivot_table.to_csv('count.csv')
#df_proquest.groupby([df_proquest['eventTimeStamp'].dt.floor('1y')]).size()

In [89]:
# Identify article year and month
df_cnt_gspx=df_lexisnexis.pivot_table(index='year_old', columns='month_old', aggfunc='size')
df_cnt_gspx=df_unique.reindex(sorted(df_cnt_gspx.columns, reverse=True), axis=1).sort_index(ascending=False)
df_cnt_gspx.to_csv('count.csv', index=False)

In [112]:
guardian_ids = list(df_api['guardianid'])
sections = []
years = []
months = []
blog_cnt = 0
for i in range(len(guardian_ids)):
    guardian_ids_separated = guardian_ids[i].split('/')
    try:
        sections.append(guardian_ids_separated[0])
        years.append(int(guardian_ids_separated[1]))
        months.append(guardian_ids_separated[2])
    except IndexError:
        print(guardian_ids[i])
    except ValueError:
        blog_cnt += 1
blog_cnt

670